# Inventory Collector
Collects data on database objects (tables and views) as well as grants on those objects.
Saves all data to a delta table

## Widget Setup
This notebook uses widgets to initialize the InventoryCollector as well as help scan a particular database easily.

In [ ]:
%sql
use catalog hive_metastore;
show databases;

In [ ]:
hmsDatabaseList = [row.databaseName for row in spark.sql('show databases').select('databaseName').collect()]
if len('hmsDatabaseList') > 1024:
    print('Warning! More than 1024 HMS databases. Picker widget will only display first 1024')
dbutils.widgets.dropdown("HMS_Database", hmsDatabaseList[0], hmsDatabaseList[0:1023], "HMS Database");
dbutils.widgets.text("Inventory_Catalog",   "hive_metastore", "Write Catalog for Inventory")
dbutils.widgets.text("Inventory_Database",   "databricks_inventory", "Write Database for Inventory")

## Collector Execution
Here are snippets on how to use the InventoryCollector

### Library Definitions

In [ ]:
from DbInventoryCollector import InventoryCollector


### Initialization
First, we must initialize the collector with a location to store its data. The .initialize() method will create the schemas and tables for the data to be stored inside.
Remember you will have to re-execute this cell every time the code for InventoryCollector is updated.

In [ ]:
#Initialize InventoryCollector
collector = InventoryCollector(spark, dbutils.widgets.get("Inventory_Catalog"), dbutils.widgets.get("Inventory_Database"))
collector.initialize()
# display(collector.get_database_inventory_summary())

### Scan of a single database
Note that there are two types of scans : objects and grants.
Each returns a pair of data: the execution id and the dataframe holding the scanned results.
All past scans are saved to an append only table. The execution_id can help you retrieve the scan as of a certain time.

With the widget code, this cell will automatically be re-run when you change the dropdown at the top.

In [ ]:
selectedDatabase = dbutils.widgets.get("HMS_Database")
(exec_id_objects, objectDF) = collector.scan_database_objects(selectedDatabase)
(exec_id_grants, grantDF) = collector.scan_database_grants(selectedDatabase)

print(f"Finished scanning both grants and objects for {selectedDatabase}. ObjectExId: {exec_id_objects} GrantExId: {exec_id_grants}")

display(grantDF)

### Auto Scan All Databases
Automatically list and scan all databases.

**Parameters:**
*rescan* -- If true, will re-scan a database even if inventory data already exists for it. If false, duplicate databases will be skipped. Default: False

In [ ]:
# collector.scan_all_databases(rescan = False)

## Results Inspection

### Summary of past executions

In [ ]:
display(collector.get_execution_history())

### Summary of all databases

In [ ]:
display(collector.get_database_inventory_summary())

### Inspect Single Database Results
There are two types of results stored. "grants" and "objects".

In [ ]:
#the "grants" result type lists out each non-inherited grant on the database and its tables and views.
db_grants = collector.get_last_results('grants', selectedDatabase)
display(db_grants)

In [ ]:
#the "objects" lists out each table and view, along with its type (managed, external, or view). If there was an error retrieving details, the error is stored. For a view the DDL is saved too.
db_objects = collector.get_last_results('objects', selectedDatabase)
display(db_objects)

In [ ]:
#You can futher aggregate the results as well
display(db_objects.groupBy('objectType').count())

## Resetting State
Upon making changes to the scanning code, you may need to reset the state. Uncomment the following cell to do so:

In [ ]:
# collector.resetAllData()